본 문서는 아래 사이트에서 제공되는 설명 및 코드를 참고하였습니다. 

* https://python.langchain.com/

# 사용자키 준비

* https://openai.com/ 에서 OPENAI_API_KEY 키를 발급받아주세요.
* https://serpapi.com/ 에서 SERPAPI_API_KEY 키를 발급받아주세요.

In [59]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."
os.environ["SERPAPI_API_KEY"] = "..."

# 패키지 설치

In [60]:
!pip install langchain  # 자연어 처리 및 기계 번역에 사용됩니다.
!pip install tiktoken   # 텍스트 토큰화에 사용됩니다.
!pip install openai     # OpenAI API와 상호작용하는 데 사용됩니다.
!pip install google-search-results # 구글 검색 결과를 가져오는 데 사용됩니다.
!pip install chromadb   # 벡터 데이터베이스를 이용하는 데 사용됩니다.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1. 모델


## 1.1. 대규모언어모델 (LLMs)

### 시작하기

In [61]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
text = "인공지능팩토리 플랫폼은 인공지능을 위한 SNS이야. 이 플랫폼을 소개하기 위한 문구를 작성해"
print(llm(text))

보자면

인공지능팩토리 플랫폼은 인공지능 기술의 연구 및 개발을 위해 딥러닝과 기계학습에 대한 연구 및 개발을 돕는 전문 SNS 플랫폼입니다. 사용자는 자신의 인공지능 프로젝트들을 공유하고 교류할 수 있고, 인공


### 반복해서 생성하기

In [62]:
llm_result = llm.generate(["개발자를 위한 챗GPT 활용 방법을 작성해"]*3)

len(llm_result.generations)

for gen_texts in llm_result.generations:
    print(gen_texts)    

[Generation(text='주세요.\n\n1. 좋은 챗GPT를 사용하기 위해서는 먼저 데이터를 잘 준비해야 합니다. 어떤 데이터를 사용할지 미리 정해놓고, 학습용 데이터에 맞춰 데이터 구성을 합니다.\n\n2. 챗GPT를 학습하기 위해서는 먼저 딥러닝 프레임워크에서 사', generation_info={'finish_reason': 'length', 'logprobs': None})]
[Generation(text=' 주세요\n\n1. GPT를 사용하기 전에 당신이 시도하고자 하는 목적을 이해합니다. GPT는 사람과 같이 사고하는 기능을 갖추고 있기 때문에, 당신이 시도하고자 하는 목적을 정확하게 정의해 주어야 합니다.\n\n2. 당신이 원하는 목적을 정한 다음,', generation_info={'finish_reason': 'length', 'logprobs': None})]
[Generation(text='보세요\n\n1. 챗GPT을 사용하기 전에 홈페이지에 제공하는 문서 등을 잘 읽어보고, 자신이 목표로 하는 개발 목적에 맞는 기능들을 이해합니다.\n\n2. 일단 챗GPT를 활용하기 위해 데이터 셋을 준비해야 합니다. 관련 키워드나 예시 문장,', generation_info={'finish_reason': 'length', 'logprobs': None})]


In [63]:
llm_result.llm_output # 사용한 모델에 대한 정보 출력

{'token_usage': {'prompt_tokens': 135,
  'completion_tokens': 767,
  'total_tokens': 902},
 'model_name': 'text-davinci-003'}

### 토큰 개수 구하기

In [64]:
llm.get_num_tokens("Hello. I am Taeyoung Kim from AIFactory")

13

In [65]:
llm.get_num_tokens("안녕하세요. 저는 인공지능팩토리의 김태영입니다.")

62

## 1.2. 채팅 모델

In [66]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)

chat([HumanMessage(content="Translate this sentence from English to Korean. I love programming.")])

AIMessage(content='저는 프로그래밍을 좋아합니다.', additional_kwargs={})

In [67]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Korean."),
    HumanMessage(content="Translate this sentence from English to Korean. I love programming.")
]

chat(messages)

AIMessage(content='저는 프로그래밍을 좋아합니다.', additional_kwargs={})

In [68]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to Korean."),
        HumanMessage(content="Translate this sentence from English to Korean. I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to Korean."),
        HumanMessage(content="Translate this sentence from English to Korean. I love artificial intelligence.")
    ],
]

result = chat.generate(batch_messages)

result

LLMResult(generations=[[ChatGeneration(text='저는 프로그래밍을 좋아합니다.', generation_info=None, message=AIMessage(content='저는 프로그래밍을 좋아합니다.', additional_kwargs={}))], [ChatGeneration(text='나는 인공지능을 사랑합니다.', generation_info=None, message=AIMessage(content='나는 인공지능을 사랑합니다.', additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 73, 'completion_tokens': 26, 'total_tokens': 99}, 'model_name': 'gpt-3.5-turbo'})

## 1.3. 텍스트 임베딩 모델

In [69]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

text = "이 문서는 시험 문서입니다."

query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text])

print(query_result)
print(doc_result)

[-0.001326117837139328, -0.008679440781908096, -0.0007732924653825813, -0.022106383853839964, -0.01967958830267894, 0.011941691317853227, -0.018379992901444812, -0.004008190989876874, -0.029201115085801053, 0.002272634430882877, -0.0036501393857251317, -0.004127541835034966, -0.010801229135066514, -0.013645752644280858, 0.010071864825225745, 0.00930271627945059]
[[-0.001326117837139328, -0.008679440781908096, -0.0007732924653825813, -0.022106383853839964, -0.01967958830267894, 0.011941691317853227, -0.018379992901444812, 0.029837649822235766, 0.03352425840134159, -0.01654994971342918, 0.007956706557174703, -0.004127541835034966, -0.010801229135066514, -0.013645752644280858, 0.010071864825225745, 0.00930271627945059]]


# 2. 프롬프트

## 2.1. 프롬프트 템플릿

In [70]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="{product}에 적합한 제품명을 추천해",
)

print(prompt.format(product="컬러 양말"))
print(prompt.format(product="스마트폰"))

컬러 양말에 적합한 제품명을 추천해
스마트폰에 적합한 제품명을 추천해


In [71]:
from langchain import PromptTemplate, FewShotPromptTemplate


# First, create the list of few shot examples.
examples = [
    {"word": "행복", "antonym": "슬픔"},
    {"word": "길다", "antonym": "짧다"},
]

# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """
word: {word}
antonym: {antonym}\n
"""
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the antonym of every input",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nAntonym:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n\n",
)

# We can now generate a prompt using the `format` method.
print(few_shot_prompt.format(input="크다"))

Give the antonym of every input


word: 행복
antonym: 슬픔




word: 길다
antonym: 짧다



Word: 크다
Antonym:


## 2.2. 채팅 프롬프트 템플릿

In [72]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language="English", output_language="Korean", text="I love programming.").to_messages())

AIMessage(content='저는 프로그래밍을 좋아합니다.', additional_kwargs={})

# 3. 체인

## 3.1. LLM과 프롬프트 템플릿을 이용한 체인

In [73]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["service"],
    template="{service}에 AI을 활용할 수 있는 방안을 작성해",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

chain.run("고객응답챗봇")

'보세요\n\n1. 빅데이터를 활용한 상담 자동화 : 데이터베이스를 관리하고 업데이트하여 고객 및 상품과 관련된 정보를 수집하여 빅데이터를 활용하여 고객과 상담 자동화를 수행할 수 있습니다.\n\n2. 시각-음성 인식 기술의 '

## 3.2. 채팅 모델과 프롬프트 템플릿을 이용한 체인


In [74]:
chat = ChatOpenAI(temperature=0)

template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt=chat_prompt)

chain.run(input_language="English", output_language="Korean", text="I love programming.")

'저는 프로그래밍을 좋아합니다.'

## 3.3 수학계산용 체인 - LLMMathChain

In [75]:
from langchain import OpenAI, LLMMathChain

llm = OpenAI(temperature=0)
llm_math = LLMMathChain(llm=llm, verbose=True)

llm_math.run("What is 13 raised to the .3432 power?")



> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?
```python
import math
print(math.pow(13, 0.3432))
```

Answer: 2.4116004626599237

> Finished chain.


'Answer: 2.4116004626599237\n'

## 3.4 프롬프트 템플릿을 이용한 수학계산용 체인 - LLMMathChain

In [76]:
from langchain.prompts.prompt import PromptTemplate

_PROMPT_TEMPLATE = """You are GPT-3, and you can't do math.

You can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.

So we hooked you up to a Python 3 kernel, and now you can execute code. If you execute code, you must print out the final answer using the print function. You MUST use the python package numpy to answer your question. You must import numpy as np.


Question: ${{Question with hard calculation.}}
```python
${{Code that prints what you need to know}}
print(${{code}})
```
```output
${{Output of your code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?

```python
import numpy as np
print(np.multiply(37593, 67))
```
```output
2518731
```
Answer: 2518731

Question: {question}"""

PROMPT = PromptTemplate(input_variables=["question"], template=_PROMPT_TEMPLATE)

llm_math = LLMMathChain(llm=llm, prompt=PROMPT, verbose=True)

llm_math.run("What is 13 raised to the .3432 power?")



> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?

```python
import numpy as np
print(np.power(13, .3432))
```

Answer: 2.4116004626599237

> Finished chain.


'Answer: 2.4116004626599237\n'

## 3.5. LLM기반 대화용 체인 - ConversationChain

In [77]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. My name is AI. What's your name?"

In [78]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. My name is AI. What's your name?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

## 3.6. 채팅 모델 기반 대화용 체인

In [79]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="Korean", text="I love programming.")

'저는 프로그래밍을 좋아합니다.'

## 3.7 스트리밍 (실시간 채팅)

In [80]:
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat = ChatOpenAI(streaming=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), verbose=True, temperature=0)
resp = chat([HumanMessage(content="Write me a song about sparkling water.")])

Verse 1:
Bubbles rising to the top
A refreshing drink that never stops
Clear and crisp, it's oh so pure
Sparkling water, I can't ignore

Chorus:
Sparkling water, oh how you shine
A taste so clean, it's simply divine
You quench my thirst, you make me feel alive
Sparkling water, you're my favorite vibe

Verse 2:
No sugar, no calories, just H2O
A drink that's good for me, don't you know
With lemon or lime, you're even better
Sparkling water, you're my forever

Chorus:
Sparkling water, oh how you shine
A taste so clean, it's simply divine
You quench my thirst, you make me feel alive
Sparkling water, you're my favorite vibe

Bridge:
You're my go-to drink, day or night
You make me feel so light
I'll never give you up, you're my true love
Sparkling water, you're sent from above

Chorus:
Sparkling water, oh how you shine
A taste so clean, it's simply divine
You quench my thirst, you make me feel alive
Sparkling water, you're my favorite vibe

Outro:
Sparkling water, you're the one for me
I'll 

# 4. 인덱스

인덱스에 사용할 파일("state_of_the_union.txt")을 다운로드 받습니다. 

In [81]:
!wget https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt

--2023-04-13 15:22:40--  https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39027 (38K) [text/plain]
Saving to: ‘state_of_the_union.txt.1’

state_of_the_union. 100%[===================>]  38.11K  --.-KB/s    in 0.002s  

2023-04-13 15:22:41 (16.0 MB/s) - ‘state_of_the_union.txt.1’ saved [39027/39027]



## 4.1. 파일기반 질의응답

In [82]:
from langchain.document_loaders import TextLoader
loader = TextLoader('state_of_the_union.txt')

from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

query = "What did the president say about Ketanji Brown Jackson"
print(index.query(query))

query = "What did the president say about Ketanji Brown Jackson"
print(index.query_with_sources(query))

 The president said that Ketanji Brown Jackson is one of the nation's top legal minds, a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. He also said that she is a consensus builder and has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.
{'question': 'What did the president say about Ketanji Brown Jackson', 'answer': " The president said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, one of the nation's top legal minds, to continue Justice Breyer's legacy of excellence, and that she has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.\n", 'sources': 'state_of_the_union.txt'}


## 4.2. 벡터DB기반 질의응답

### stuff 방식 - 기본

In [83]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator

In [84]:
with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

query = "What did the president say about Justice Breyer"
docs = docsearch.get_relevant_documents(query)

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
query = "What did the president say about Justice Breyer"
chain.run(input_documents=docs, question=query)

' The president said that Justice Breyer is an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court, and thanked him for his service.'

### stuff 방식 - 사용자 정의 프롬프트 템플릿

In [85]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in Korean:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' 대통령은 스티븐 브라이어 재판관에 대해 말했다, 그는 군인, 헌법학자 및 미국 최고법원의 퇴임 재판관이며 이 나라를 섬기기 위해 삶을 헌신한 사람이라고 기념하고 싶다고 말했다.'}

### map-rerank 방식 - 기본

In [86]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_rerank", return_intermediate_steps=True)

query = "What did the president say about Justice Breyer"
results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

results["output_text"]

results["intermediate_steps"]

[{'answer': ' The President thanked Justice Breyer for his service and honored him for dedicating his life to serve the country.',
  'score': '100'},
 {'answer': ' This document does not answer the question', 'score': '0'},
 {'answer': ' This document does not answer the question', 'score': '0'},
 {'answer': ' This document does not answer the question', 'score': '0'}]

### map-rerank - 사용자 정의 프롬프트 템플릿

In [87]:
from langchain.output_parsers import RegexParser

output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer In Italian: [answer here]
Score: [score between 0 and 100]

Begin!

Context:
---------
{context}
---------
Question: {question}
Helpful Answer In Korean:"""
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser,
)

chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_rerank", return_intermediate_steps=True, prompt=PROMPT)
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': [{'answer': ' 대통령은 스티븐 브라이어 재판관에 대해 무엇을 말했습니까?',
   'score': '100'},
  {'answer': ' 대통령은 브라이어 재판관에 대해 무엇을 말했습니까?', 'score': '0'},
  {'answer': ' 대통령은 브라이어 재판관에 대해 무엇을 말했습니까?', 'score': '0'},
  {'answer': ' 대통령은 저스티스 브라이어에 대해 무엇을 말했습니까?', 'score': '0'}],
 'output_text': ' 대통령은 스티븐 브라이어 재판관에 대해 무엇을 말했습니까?'}

## 4.3. RetrievalQA 체인을 이용한 검색 기반 질의응답

In [88]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

from langchain.document_loaders import TextLoader
loader = TextLoader("state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever())

query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

" The president said that Ketanji Brown Jackson is one of the nation's top legal minds and a consensus builder who has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. He said she is a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers."

## 4.4. RetrievalQA 체인과 프롬프트 템플릿을 이용한 검색 기반 질의응답

In [89]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in Korean:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)

query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

" 대통령은 케탄지 브론 잭슨에 대해 무엇을 말했나요? \nAnswer in English: The President said that Judge Ketanji Brown Jackson is one of the nation's top legal minds, a consensus builder, and has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."

## 4.5. 요약체인과 문서분석체인을 이용한 요약

In [90]:
with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()

from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0)
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")

from langchain.chains import AnalyzeDocumentChain

summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

summarize_document_chain.run(state_of_the_union)

" In this speech, President Biden addresses the American people and the world, discussing the recent aggression of Russia's Vladimir Putin in Ukraine and the US response. He outlines economic sanctions and other measures taken to hold Putin accountable, and announces the US Department of Justice's task force to go after the crimes of Russian oligarchs. He also outlines plans to invest in America, modernize infrastructure, and promote environmental justice, and proposes a plan to fight inflation. He calls for the continued combat of COVID-19, and proposes a Unity Agenda for the Nation. He also provides assistance to lower-income veterans and leads the Cancer Moonshot. He concludes by emphasizing the strength of the nation and the need to come together to overcome the challenges of our time."

## 4.6. 질의응답체인과 문서분석체인을 이용한 질의응답


In [91]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm, chain_type="map_reduce")

qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

qa_document_chain.run(input_document=state_of_the_union, question="what did the president say about justice breyer?")

' The president thanked Justice Breyer for his service.'

# 5. 메모리

## 5.1. 메모리를 이용한 챗봇

In [92]:
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain, PromptTemplate

template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(
    llm=OpenAI(), 
    prompt=prompt, 
    verbose=True, 
    memory=memory,
)

llm_chain.predict(human_input="Hi there my friend")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.


Human: Hi there my friend
Chatbot:

> Finished chain.


' Hi there, how are you doing today?'

In [93]:
llm_chain.predict(human_input="Not to bad - how are you?")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Human: Hi there my friend
AI:  Hi there, how are you doing today?
Human: Not to bad - how are you?
Chatbot:

> Finished chain.


" I'm doing great, thanks for asking!"

# 6. 에이전트

## 6.1. 구글 검색 도구

In [94]:
from langchain.utilities import SerpAPIWrapper

search = SerpAPIWrapper()
search.run("Obama's first name?")

'Barack Hussein Obama II'

In [95]:
params = {
    "engine": "bing",
    "gl": "us",
    "hl": "en",
}
search = SerpAPIWrapper(params=params)

search.run("Obama's first name?")

'Barack Hussein Obama II is an American former politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-Ame…New content will be added above the current area of focus upon selectionBarack Hussein Obama II is an American former politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president of the United States. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer before holding public office.Wikipediabarackobama.com'

## 6.2. 구글 검색 도구와 수학 도구를 이용한 에이전트

In [96]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")



> Entering new AgentExecutor chain...
 I need to find the temperature first, then use the calculator to raise it to the .023 power.
Action: Search
Action Input: "High temperature in SF yesterday"
Observation: High: 59ºf @3:40 PM Low: 48.02ºf @5:56 AM Approx. Precipitation / Rain Total: in. 1hr.
Thought: I need to use the calculator to raise the temperature to the .023 power.
Action: Calculator
Action Input: 59^.023
Observation: Answer: 1.0983217810400328

Thought: I now know the final answer.
Final Answer: The high temperature in SF yesterday in Fahrenheit raised to the .023 power is 1.0983217810400328.

> Finished chain.


'The high temperature in SF yesterday in Fahrenheit raised to the .023 power is 1.0983217810400328.'

## 6.3. 채팅 모델을 이용한 에이전트

In [97]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")



> Entering new AgentExecutor chain...
Thought: I need to use a search engine to find Olivia Wilde's boyfriend and a calculator to raise his age to the 0.23 power.
Action:
```
{
  "action": "Search",
  "action_input": "Olivia Wilde boyfriend"
}
```

Observation: Olivia Wilde started dating Harry Styles after ending her years-long engagement to Jason Sudeikis — see their relationship timeline.
Thought:Now I need to use a calculator to raise Harry Styles' age to the 0.23 power.
Action:
```
{
  "action": "Calculator",
  "action_input": "pow(27, 0.23)"
}
```

Observation: Answer: 2.1340945944237553

Thought:I now know the final answer.
Final Answer: 2.1340945944237553

> Finished chain.


'2.1340945944237553'